In [1]:
import h5py
import pandas as pd
from policyengine_uk import Microsimulation

weight_data = h5py.File("constituencies/weights.h5", "r")
constituencies = pd.read_csv("constituencies/constituencies.csv")

In [2]:
baseline = Microsimulation()

hnet_24 = baseline.calculate("real_household_net_income", 2024)
hnet_29 = baseline.calculate("real_household_net_income", 2029)

In [3]:
change = hnet_29 - hnet_24

In [4]:
from constituencies.charts import constituencies as hex_con

In [19]:
import plotly.express as px
import numpy as np

weights = weight_data["weight"]


weighted_gain = np.dot(weights, change)
weighted_original_income = np.dot(weights, hnet_24)
weighted_rel_gain = weighted_gain / weighted_original_income

df = pd.DataFrame()
df["Average real change (%)"] = weighted_rel_gain > np.median(weighted_rel_gain)
df["Constituency"] = constituencies.name
df["Code"] = constituencies.code

df["y"] = hex_con.set_index("code").loc[df.Code].y.values
df["x"] = hex_con.set_index("code").loc[df.Code].x.values
df.x = df.x + (df.y % 2 == 0) * 0.5

In [20]:
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="Average real change (%)",
    hover_data=["Constituency"]
)

from policyengine_core.charts import *

fig.update_layout(
    xaxis_range=[-5, 45],
    yaxis_range=[-5, 45],
    height=580,
    width=700,
)
# Set zero to white
#fig.update_coloraxes(cmax=-df["Average real change"].min(), cmin=df["Average real change"].min())

# Set marker to hexagon
fig.update_traces(marker=dict(symbol="hexagon", size=11))

fig.update_layout(
    title="Baseline forecast for real household net income growth (2024-29)",
    xaxis_tickvals=[],
    xaxis_title="",
    yaxis_tickvals=[],
    yaxis_title="",
    # No bg
    plot_bgcolor="rgba(0,0,0,0)",
    legend_title="Average household net income change",
    # Set color axis to %
    title_font_color="black",
    legend_title_font_color="black"
)

In [7]:
df[["Code", "Constituency", "Average real change (%)"]].to_csv("constituency_baseline_forecast.csv", index=False)

In [8]:
df["Average real change (%)"].describe()

count    650.000000
mean       2.488307
std        1.017210
min       -1.600000
25%        1.800000
50%        2.400000
75%        3.000000
max        5.700000
Name: Average real change (%), dtype: float64

In [9]:
df.sort_values("Average real change (%)")

,Average real change (%),Constituency,Code,y,x
619,-1.6,Na h-Eileanan an Iar,S14000027,40,9.5
277,-0.8,Great Yarmouth,E14000717,13,32.0
34,-0.6,Blackley and Broughton,E14000571,23,13.0
41,-0.3,Burnley,E14000609,27,16.0
25,0.1,Sunderland Central,E14000982,31,21.0
...,...,...,...,...,...
392,5.6,Westminster North,E14001036,10,22.5
380,5.6,Richmond Park,E14000896,6,21.5
369,5.6,Kensington,E14000768,8,23.5
364,5.6,Hornsey and Wood Green,E14000752,11,23.0
